In [1]:
from datetime import datetime

from gym.wrappers import FlattenObservation, Monitor, TimeLimit
from stable_baselines3 import TD3

from environments.ocelot import ARESEAOcelot
from environments.simulation import ARESEACheetah

initializing ocelot...


In [2]:
run_name = "pretty-jazz-258"
env_name = "ocelot"

In [3]:
model = TD3.load(f"models/{run_name}")
model

In [4]:
timestr = datetime.now().strftime("%Y%m%d%H%M%S")

env = ARESEACheetah() if env_name == "cheetah" else ARESEAOcelot()
env = TimeLimit(env, max_episode_steps=50)
env = FlattenObservation(env)
env = Monitor(env, f"recordings/test_{run_name}_{env_name}_{timestr}", video_callable=lambda _: True)
env

/usr/local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


<Monitor<FlattenObservation<TimeLimit<ARESEAOcelot instance>>>>

In [5]:
observation = env.reset()

In [6]:
done = False
# observation = env.reset()
# observation, _, _, _ = env.step(np.array([0.0, 0.0, 0.0, 0.0, 0.0]))
while not done:
    action, _ = model.predict(observation)
    observation, reward, done, info = env.step(action)
env.close()

In [7]:
env.actuators

array([ 7.2868567e+00, -5.2452955e+00, -4.8131526e-02, -1.0328693e-03,
       -7.2792039e-04], dtype=float32)